# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import random
from model import load_data, get_analytics
from cytoself_custom import plot_umap, calc_umap_embvec, plot_feature_spectrum_from_image, arrange_plots
from synthetic_multiplexing import multiplex
from metrics import plot_boostrapping, calc_bootstrapping, plot_metrics, calc_reconstruction_error
from explainability import generate_images, plot_clusters_distance_heatmap
from utils import get_colors_dict
from sklearn.metrics.pairwise import euclidean_distances
import seaborn as sns

from tensorflow.compat.v1.keras.models import Model


import os
import config
from config import SEED, PRETRAINED_MODEL_PATH, MODEL_FOLDER, METRICS_FOLDER, METRICS_MATCH_PATH, METRICS_RANDOM_PATH, MARKERS

np.random.seed(SEED)
random.seed(SEED)

%reload_ext autoreload
%autoreload 2
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



# Logs

In [2]:
import logging
log_file_path = f"./logs/fc_outputs_log.log"
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(levelname)s %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    handlers=[
        logging.FileHandler(log_file_path),
        logging.StreamHandler()
    ])

logging.info("init")

2022-12-19 12:13:00 INFO init


## Config

In [3]:
logging.info(f"Set config")

MODEL_PATH = os.path.join(MODEL_FOLDER, "MODEL18_model_weights.0040.h5")

# input_folders =["./data/processed/220629_neurons_12markers_unstressed",
#                  "./data/processed/220707_neurons_12markers_stressed"]
input_folders =[ "./data/processed/220814_neurons",
                 "./data/processed/220818_neurons",
                 "./data/processed/220831_neurons",
                 "./data/processed/220908",
                 "./data/processed/220914"]
cytoself_model_path = PRETRAINED_MODEL_PATH
neuroself_model_path = MODEL_PATH
markers = MARKERS #['G3BP1', 'PURA', 'FMRP', 'DAPI'] #MARKERS
n_samples = 10

labels_unique = np.array(['ANXA11_unstressed', 'CD41_unstressed', 'CLTC_unstressed',
 'Calreticulin_unstressed', 'DAPI_unstressed', 'DCP1A_unstressed',
 'FMRP_unstressed', 'FUS_unstressed', 'G3BP1_unstressed', 'GM130_unstressed',
 'KIF5A_unstressed', 'LAMP1_unstressed', 'NEMO_unstressed', 'NONO_unstressed',
 'Nucleolin_unstressed', 'PEX14_unstressed', 'PML_unstressed',
 'PSD95_unstressed', 'PURA_unstressed', 'Phalloidin_unstressed',
 'SNCA_unstressed', 'SQSTM1_unstressed', 'TDP43_unstressed',
 'TIA1_unstressed', 'TOMM20_unstressed', 'mitotracker_unstressed'])

2022-12-19 12:13:00 INFO Set config


## Load Data

In [4]:
logging.info(f"Loading data")
images, labels, labels_changepoints, markers_order = load_data(input_folders, condition_l=True, type_l=False,
                                                               cell_lines_include=["WT"], 
                                                               split_by_set=True,
                                                               # split_by_set=False,
                                                               set_type="test",
                                                            # set_type="test",
                                                               markers=markers,
                                                               conds_include=['unstressed'])
                                                            #    split_by_set_include=[("WT", "unstressed")])

logging.info(f"Labels: {np.unique(labels)}")

2022-12-19 12:13:00 INFO Loading data


#########################################################
########### Splitting by set! (test) #############
#########################################################
Input folder: ./data/processed/220814_neurons
Input subfolder: CD41
Filepath: ./data/processed/220814_neurons/WT/unstressed/CD41/220807_iNDI_WT_unstressed-DAPI_CD41_NONO_TDP43-01_WT.tif_processed
Input subfolder: CLTC
Filepath: ./data/processed/220814_neurons/WT/unstressed/CLTC/220808_iNDI_WT_unstressed-DAPI_Phalloidin_PSD95_CLTC-01_WT.tif_processed
Input subfolder: DAPI
Filepath: ./data/processed/220814_neurons/WT/unstressed/DAPI/220805_iNDI_WT_unstressed-DAPI_G3BP1_TIA1_KIF5A-01_WT.tif_processed
Filepath: ./data/processed/220814_neurons/WT/unstressed/DAPI/220807_iNDI_WT_unstressed-DAPI_CD41_NONO_TDP43-01_WT.tif_processed
Filepath: ./data/processed/220814_neurons/WT/unstressed/DAPI/220808_iNDI_WT_unstressed-DAPI_PURA_SQSTM1_FMRP-01_WT.tif_processed
Filepath: ./data/processed/220814_neurons/WT/unstressed/DAPI/220808_iNDI_

2022-12-19 12:13:05 INFO Labels: ['ANXA11_unstressed' 'CD41_unstressed' 'CLTC_unstressed'
 'Calreticulin_unstressed' 'DAPI_unstressed' 'DCP1A_unstressed'
 'FMRP_unstressed' 'FUS_unstressed' 'G3BP1_unstressed' 'GM130_unstressed'
 'KIF5A_unstressed' 'LAMP1_unstressed' 'NEMO_unstressed' 'NONO_unstressed'
 'Nucleolin_unstressed' 'PEX14_unstressed' 'PML_unstressed'
 'PSD95_unstressed' 'PURA_unstressed' 'Phalloidin_unstressed'
 'SNCA_unstressed' 'SQSTM1_unstressed' 'TDP43_unstressed'
 'TIA1_unstressed' 'TOMM20_unstressed' 'mitotracker_unstressed']


Input subfolder: PML
Filepath: ./data/processed/220914/WT/unstressed/PML/220910_iNDI_WT_unstressed-DAPI_PML_PEX14-_WT.tif_processed
Input subfolder: Syto12
Skipping Syto12
(805, 100, 100, 2) (805, 1)


## Get Analytics

In [5]:
logging.info(f"Getting analytics...")
analytics_neuroself = get_analytics(images,labels, model_path=neuroself_model_path)


2022-12-19 12:13:05 INFO Getting analytics...


X, y:
(805, 100, 100, 2) (805,)
DAPI_unstressed            240
Phalloidin_unstressed       37
mitotracker_unstressed      29
Nucleolin_unstressed        29
FUS_unstressed              29
GM130_unstressed            27
PML_unstressed              24
PEX14_unstressed            24
TDP43_unstressed            23
CD41_unstressed             23
NONO_unstressed             23
TIA1_unstressed             21
CLTC_unstressed             21
ANXA11_unstressed           21
Calreticulin_unstressed     21
PSD95_unstressed            21
LAMP1_unstressed            21
SNCA_unstressed             21
KIF5A_unstressed            21
G3BP1_unstressed            21
NEMO_unstressed             19
DCP1A_unstressed            19
SQSTM1_unstressed           18
PURA_unstressed             18
FMRP_unstressed             18
TOMM20_unstressed           16
dtype: int64
Creating the model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2022-12-19 12:13:05 WARNING From /home/labs/hornsteinlab/sagyk/anaconda3/envs/cytoself/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2022-12-19 12:13:11 WARNING From /home/labs/hornsteinlab/sagyk/anaconda3/envs/cytoself/lib/python3.7/site-packages/tensorflow_core/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




All files will be saved at model_outputs/


Loading weights


2022-12-19 12:13:15.357839: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/share/lsf/10.1/linux3.10-glibc2.17-x86_64/lib
2022-12-19 12:13:15.357875: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-19 12:13:15.357903: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (access.wexac.weizmann.ac.il): /proc/driver/nvidia/version does not exist
2022-12-19 12:13:15.358708: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2022-12-19 12:13:15.477132: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2022-12-19 12:13:15.479049: I tensorflow/compiler/xla

./models/MODEL18_model_weights.0040.h5
is loaded.
Ground truth:
['ANXA11_unstressed' 'CD41_unstressed' 'CLTC_unstressed'
 'Calreticulin_unstressed' 'DAPI_unstressed' 'DCP1A_unstressed'
 'FMRP_unstressed' 'FUS_unstressed' 'G3BP1_unstressed' 'GM130_unstressed'
 'KIF5A_unstressed' 'LAMP1_unstressed' 'NEMO_unstressed' 'NONO_unstressed'
 'Nucleolin_unstressed' 'PEX14_unstressed' 'PML_unstressed'
 'PSD95_unstressed' 'PURA_unstressed' 'Phalloidin_unstressed'
 'SNCA_unstressed' 'SQSTM1_unstressed' 'TDP43_unstressed'
 'TIA1_unstressed' 'TOMM20_unstressed' 'mitotracker_unstressed']


## Build model

In [6]:
logging.info(f"Build model...")
model_fc = Model(analytics_neuroself.model.img_input, analytics_neuroself.model.fc_outputs[::-1])

2022-12-19 12:13:18 INFO Build model...


## Predict

In [7]:
logging.info(f"Predict init...")

##### TAKE ALL!
n_samples = len(analytics_neuroself.data_manager.test_data)
#####

indexes = np.random.choice(np.arange(len(analytics_neuroself.data_manager.test_data)), n_samples, replace=False)

data = analytics_neuroself.data_manager.test_data[indexes]
labels = analytics_neuroself.data_manager.test_label[indexes]
logging.info(f"{len(data)}, {len(labels)}, {indexes}")

labels_true = labels.reshape(-1,)

logging.info(f"Predict...")
predictions = model_fc.predict(data, batch_size=min(n_samples, analytics_neuroself.model.batch_size * 4), verbose=1)
predictions = [labels_unique[np.argmax(p, 1)] for p in predictions]
pred1, pred2 = predictions[0], predictions[1]
# if not isinstance(prediction, list):
#     prediction = [prediction]
# if savepath:
#     if savepath == "default":
#         savepath = self.savepath_dict["emb"]
#     [
#         np.save(os.path.join(savepath, f"{filename}{i + 1}.npy"), v)
#         for i, v in enumerate(embvec)
#     ]
# if do_return:
#     return embvec
# else:
#     self.embvec = embvec



logging.info(f"True labels: {labels_true}")
logging.info(f"Predicted labels (fc1): {pred1}")
logging.info(f"Predicted labels (fc2): {pred2}")

diffs1 = [(i,x[0],x[1]) for i, x in enumerate(zip(labels_true,pred1)) if x[0]!=x[1]]
diffs2 = [(i,x[0],x[1]) for i, x in enumerate(zip(labels_true,pred2)) if x[0]!=x[1]]

logging.info(f"Diff 1 ({len(diffs1)}, {len(diffs1)*100/len(labels_true)}): {diffs1}")
logging.info(f"Diff 2 ({len(diffs2)}, {len(diffs2)*100/len(labels_true)}): {diffs2}")

2022-12-19 12:13:18 INFO Predict init...
2022-12-19 12:13:18 INFO 805, 805, [613 508 143 686  95 529 458 761 723 543  37 793 776 628  47 361 100  68
  87 574 433 471 141 482 462  81 337 328   3 489  77 306 674 502 709 188
 439 245 770 261  63 513 268  35 558  80 257 755  36 289 636   0 449 193
 754 149 407 282 349 530 512 646 226 325 252 557 758 376 310 494 614 241
 437  16 496  38 290 342 209 720 627 260  18 749 736  55 254 479 683  82
 341 484 113 286 161 278 295 255  98 371 607 107  48 589 753 467 464 488
 344 612 673  12 493 264 405 364 228 274  19 606  43 329  17  61 555 338
 744 105 677  21 586 635 710 604 375 385 656 540   8 692 140 460 217 146
 763  11 682 490 104 455 495 301 747  25 721 700 400 561 804 281 708 363
 432 219 443 110 547 746 620 114 169 155 302 312 790 600 500 444 691 174
 605 738 775 112  92 179 213 799 292   7 135 616  62  14 262 200 352  94
 399 123 751 483 198 632 288 629  70 779  30  86 622 781 597 509 372 654
 528 242 789 125 565 581 427 671 394 170 164 657

805/805 [==============================] - 3s 4ms/sample


2022-12-19 12:13:21 INFO True labels: ['Nucleolin_unstressed' 'DAPI_unstressed' 'FMRP_unstressed'
 'DAPI_unstressed' 'DAPI_unstressed' 'DAPI_unstressed'
 'Calreticulin_unstressed' 'PEX14_unstressed' 'DAPI_unstressed'
 'DAPI_unstressed' 'CLTC_unstressed' 'PML_unstressed' 'PEX14_unstressed'
 'Nucleolin_unstressed' 'DAPI_unstressed' 'DAPI_unstressed'
 'DAPI_unstressed' 'DAPI_unstressed' 'DAPI_unstressed' 'FUS_unstressed'
 'TOMM20_unstressed' 'Calreticulin_unstressed' 'FMRP_unstressed'
 'DAPI_unstressed' 'Calreticulin_unstressed' 'DAPI_unstressed'
 'DAPI_unstressed' 'TIA1_unstressed' 'CD41_unstressed' 'DAPI_unstressed'
 'DAPI_unstressed' 'TDP43_unstressed' 'mitotracker_unstressed'
 'DAPI_unstressed' 'DAPI_unstressed' 'NONO_unstressed' 'ANXA11_unstressed'
 'PURA_unstressed' 'PEX14_unstressed' 'Phalloidin_unstressed'
 'DAPI_unstressed' 'DAPI_unstressed' 'Phalloidin_unstressed'
 'CLTC_unstressed' 'FUS_unstressed' 'DAPI_unstressed'
 'Phalloidin_unstressed' 'GM130_unstressed' 'CLTC_unstressed'


In [8]:
logging.info(f"Done.")

2022-12-19 12:13:21 INFO Done.
